In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tfredrick112","key":"419ebbf3e48d77719f5f5ded943690aa"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d tfredrick112/labels-all-aacb

  0% 0.00/6.03M [00:00<?, ?B/s]
100% 6.03M/6.03M [00:00<00:00, 55.7MB/s]


In [ ]:
! unzip labels-all-aacb.zip

Archive:  labels-all-aacb.zip
  inflating: BMNC_ADT.csv            


In [ ]:
!gdown https://drive.google.com/uc?id=1VaY5GzetBlNbElwDMfXTDXaSpk4j-VbS

Downloading...
From: https://drive.google.com/uc?id=1VaY5GzetBlNbElwDMfXTDXaSpk4j-VbS
To: /content/RNA_features_scaledata_BMNC.csv
1.21GB [00:07, 172MB/s]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from time import time
import xgboost
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("RNA_features_scaledata_BMNC.csv", index_col=0)

In [ ]:
df.shape

(30672, 2000)

In [ ]:
df.head()

,HES4,ISG15,TNFRSF18,TNFRSF4,MRPL20,ATAD3A,PRDM16,SMIM1,ACOT7,ESPN,PARK7,ENO1,SPSB1,RBP7,APITD1,SRM,KAZN,DDI2,MRTO4,PLA2G2D,UBXN10,C1QA,C1QC,C1QB,ID3,PITHD1,RHD,RHCE,AUNIP,STMN1,ZNF593,ZNF683,HMGN2,NUDC,IFI6,ATPIF1,RCC1,MECR,SNRNP40,CCDC28B,...,RP11-672A2.6,NNMT,TTC36,RP11-138I18.2,CCDC3,RET,PRKG1,ADRA2A,GALNT8,AICDA,KRT5,RP11-341G23.4,CABP1,THRIL,LECT1,POU4F1,ITGBL1,TRAV29DV5,CTD-2509G16.5,AC005480.1,IGHV1-2,SCG5,RP11-798K3.3,ARHGDIG,RP11-510M2.2,VAT1L,CTC-297N7.1,RP11-1072C15.4,HOXB5,DOK6,SRXN1,SLPI,CTXN1,IGLV9-49,IGLV5-48,IGLV7-43,IGLV3-27,SLC16A8,RP3-333H23.8,FAM19A5
a_AAACCTGAGCTTATCG-1,-0.078757,-0.390445,-0.111718,-0.153079,0.617982,-0.176082,-0.021664,2.939813,4.979018,-0.05104,0.266658,1.808219,-0.044829,-0.286194,-0.116317,1.614860,-0.026955,3.164261,-0.223353,-0.022504,-0.060879,-0.069561,-0.020838,-0.036406,-0.152312,-0.339,-0.061007,-0.083792,-0.074683,1.291601,1.676841,-0.051874,1.660152,-0.434871,-0.332308,0.270333,-0.177524,4.067965,1.479162,4.026699,...,-0.015264,-0.017265,-0.011534,-0.015621,-0.013248,-0.014978,-0.01058,-0.022694,-0.012262,-0.014159,-0.016521,-0.015312,-0.012288,-0.015025,-0.016377,-0.014804,-0.012761,-0.015963,-0.013583,-0.010785,-0.017064,-0.014877,-0.014947,-0.010946,-0.014964,-0.015287,-0.011493,-0.014427,-0.022445,-0.016005,-0.012595,-0.018368,-0.013759,-0.016088,-0.01393,-0.022088,-0.02177,-0.015401,-0.016619,-0.014287
a_AAACCTGAGGTGGGTT-1,-0.078757,-0.390445,-0.111718,-0.153079,-0.546178,-0.176082,-0.021664,-0.181047,-0.126231,-0.05104,-0.714860,-0.735061,-0.044829,-0.286194,-0.116317,-0.451785,-0.026955,-0.171434,-0.223353,-0.022504,-0.060879,-0.069561,-0.020838,-0.036406,-0.152312,-0.339,-0.061007,-0.083792,-0.074683,-0.430186,-0.295914,-0.051874,-1.111235,-0.434871,-0.332308,-0.692979,-0.177524,-0.146120,-0.331998,-0.213503,...,-0.015264,-0.017265,-0.011534,-0.015621,-0.013248,-0.014978,-0.01058,-0.022694,-0.012262,-0.014159,-0.016521,-0.015312,-0.012288,-0.015025,-0.016377,-0.014804,-0.012761,-0.015963,-0.013583,-0.010785,-0.017064,-0.014877,-0.014947,-0.010946,-0.014964,-0.015287,-0.011493,-0.014427,-0.022445,-0.016005,-0.012595,-0.018368,-0.013759,-0.016088,-0.01393,-0.022088,-0.02177,-0.015401,-0.016619,-0.014287
a_AAACCTGAGTACATGA-1,-0.078757,-0.390445,-0.111718,-0.153079,-0.546178,-0.176082,-0.021664,-0.181047,-0.126231,-0.05104,-0.714860,-0.735061,-0.044829,-0.286194,-0.116317,-0.451785,-0.026955,-0.171434,-0.223353,-0.022504,-0.060879,-0.069561,-0.020838,-0.036406,-0.152312,-0.339,-0.061007,-0.083792,-0.074683,-0.430186,-0.295914,-0.051874,-1.111235,-0.434871,3.807554,-0.692979,-0.177524,-0.146120,-0.331998,-0.213503,...,-0.015264,-0.017265,-0.011534,-0.015621,-0.013248,-0.014978,-0.01058,-0.022694,-0.012262,-0.014159,-0.016521,-0.015312,-0.012288,-0.015025,-0.016377,-0.014804,-0.012761,-0.015963,-0.013583,-0.010785,-0.017064,-0.014877,-0.014947,-0.010946,-0.014964,-0.015287,-0.011493,-0.014427,-0.022445,-0.016005,-0.012595,-0.018368,-0.013759,-0.016088,-0.01393,-0.022088,-0.02177,-0.015401,-0.016619,-0.014287
a_AAACCTGCAAACCTAC-1,-0.078757,-0.390445,7.734547,-0.153079,1.573656,-0.176082,-0.021664,-0.181047,-0.126231,-0.05104,-0.714860,1.574608,-0.044829,-0.286194,-0.116317,-0.451785,-0.026955,-0.171434,-0.223353,-0.022504,-0.060879,-0.069561,-0.020838,-0.036406,-0.152312,-0.339,-0.061007,-0.083792,-0.074683,-0.430186,-0.295914,-0.051874,-1.111235,-0.434871,-0.332308,-0.692979,-0.177524,-0.146120,-0.331998,-0.213503,...,-0.015264,-0.017265,-0.011534,-0.015621,-0.013248,-0.014978,-0.01058,-0.022694,-0.012262,-0.014159,-0.016521,-0.015312,-0.012288,-0.015025,-0.016377,-0.014804,-0.012761,-0.015963,-0.013583,-0.010785,-0.017064,-0.014877,-0.014947,-0.010946,-0.014964,-0.015287,-0.011493,-0.014427,-0.022445,-0.016005,-0.012595,-0.018368,-0.013759,-0.016088,-0.01393,-0.022088,-0.02177,-0.015401,-0.016619,-0.014287
a_AAACCTGCAAGGTGTG-1,-0.078757,-0.390445,-0.111718,-0.153079,-0.546178,-0.176082,-0.021664,-0.181047,-0.126231,-0.05104,1.322673,-0.735061,-0.044829,5.185558,-0.116317,

In [ ]:
# The ADT file contains the labels, i.e. the expression levels for 25 proteins (for each cell)
df_labels = pd.read_csv("BMNC_ADT.csv")

# Transpose to get the dataset in the correct format
df_labels = df_labels.transpose()

# Set the first row as the column headers
df_labels.columns = df_labels.iloc[0]
df_labels = df_labels.drop("Unnamed: 0")

# Visualize
df_labels.head()

Unnamed: 0,CD11a,CD11c,CD123,CD127-IL7Ra,CD14,CD16,CD161,CD19,CD197-CCR7,CD25,CD27,CD278-ICOS,CD28,CD3,CD34,CD38,CD4,CD45RA,CD45RO,CD56,CD57,CD69,CD79b,CD8a,HLA.DR
a_AAACCTGAGCTTATCG-1,1.43835,0.912286,0.342896,0.460154,0.363419,0.900484,0.363419,0.256314,0.422575,0.21004,1.13254,0.597761,0.581561,1.21387,1.49888,1.97321,0.88854,1.21387,0.21004,0.278673,0.760534,0.732842,0.136347,0.826584,2.02851
a_AAACCTGAGGTGGGTT-1,2.94867,0.33722,0.154525,1.98095,0.210216,0.643093,1.97633,0.262968,0.0647021,0.0328743,1.74262,1.01931,1.01931,3.32275,0.125471,0.660509,3.267,0.711006,0.531706,0.759076,0.313077,0.182758,0.262968,0.406337,0.727287
a_AAACCTGAGTACATGA-1,2.1405,0.485698,0.0606537,1.30531,0.223242,0.446465,0.223242,0.171928,0.223242,0,3.24891,0.916587,1.33865,4.12863,0.223242,1.179,3.63312,2.29045,0.0606537,0.117838,0.754033,0.40563,0,0.865283,0.559828
a_AAACCTGCAAACCTAC-1,2.82035,0.706906,0.115421,1.33867,0.218888,0.477309,0.381797,0.218888,0.257453,0.136989,2.76335,1.44219,1.27236,3.61406,0.158101,1.01588,3.45737,0.618652,1.10077,0.0241747,0.430693,0.446474,0.0477787,0.550475,0.90347
a_AAACCTGCAAGGTGTG-1,3.81884,2.83033,0.500528,0.281259,2.18643,0.396891,0.239539,0.281259,0.225237,0.102796,0.871045,0.321307,0.25364,1.28048,0.396891,2.14082,1.20769,0.671078,0.623863,0.0527184,0.553778,1.40746,0.118947,0.671078,1.86398


In [ ]:
df_labels.shape

(30672, 25)

In [ ]:
X = np.array(df)
y = np.array(df_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# These matrices store the predictions
y_train_pred = np.zeros(y_train.shape)
y_test_pred = np.zeros(y_test.shape)

# Random forest models
models_list = [None] * 25

# Model training
for j in range(25):
    start = time()
    nn_reg = MLPRegressor(n_estimators=10)
    nn_reg.fit(X_train, y_train[:, j])

    # Predictions
    y_train_pred[:, j] = nn_reg.predict(X_train)
    y_test_pred[:, j] = nn_reg.predict(X_test)
    
    # R2 score for the model (training data)
    print("Training R2 score for label {} = {}".format(j, r2_score(y_train[:, j], y_train_pred[:, j])))

    # R2 score for the model (test data)
    print("Test R2 score for label {} = {}".format(j, r2_score(y_test[:, j], y_test_pred[:, j])))

    # MSE for the model (training data)
    print("Training MSE for label {} = {}".format(j, mean_squared_error(y_train[:, j], y_train_pred[:, j])))

    # MSE for the model (test data)
    print("Test MSE for label {} = {}".format(j, mean_squared_error(y_test[:, j], y_test_pred[:, j])))

    end = time()
    print("Time taken for the {}th column = {} minutes".format(j, (end-start)/60))
    print()

    # Store the model
    models_list[j] = nn_reg

Training R2 score for label 0 = 0.9639282453598518
Test R2 score for label 0 = 0.7976472631208087
Training MSE for label 0 = 0.017097440481228854
Test MSE for label 0 = 0.09631547750672931
Time taken for the 0th column = 2.3484222133954367 minutes

Training R2 score for label 1 = 0.9845516989314378
Test R2 score for label 1 = 0.9118201343494249
Training MSE for label 1 = 0.01242874045813509
Test MSE for label 1 = 0.07200816237786764
Time taken for the 1th column = 2.3896349668502808 minutes

Training R2 score for label 2 = 0.9534122284051069
Test R2 score for label 2 = 0.7674365993046204
Training MSE for label 2 = 0.010219119892604746
Test MSE for label 2 = 0.05073491326525528
Time taken for the 2th column = 4.148572329680125 minutes

Training R2 score for label 3 = 0.9487784562315751
Test R2 score for label 3 = 0.7234368283632164
Training MSE for label 3 = 0.02030056076174845
Test MSE for label 3 = 0.11137730217926593
Time taken for the 3th column = 2.3154404838879903 minutes

Trainin

In [ ]:

# These matrices store the predictions
y_train_pred = np.zeros(y_train.shape)
y_test_pred = np.zeros(y_test.shape)


In [ ]:
j = 8
start = time()
nn_reg = MLPRegressor(hidden_layer_sizes=(2500, 1000, 500))
nn_reg.fit(X_train, y_train[:, j])

end = time()

In [ ]:
# Predictions
y_train_pred[:, j] = nn_reg.predict(X_train)
y_test_pred[:, j] = nn_reg.predict(X_test)

# R2 score for the model (training data)
print("Training R2 score for label {} = {}".format(j, r2_score(y_train[:, j], y_train_pred[:, j])))

# R2 score for the model (test data)
print("Test R2 score for label {} = {}".format(j, r2_score(y_test[:, j], y_test_pred[:, j])))

# MSE for the model (training data)
print("Training MSE for label {} = {}".format(j, mean_squared_error(y_train[:, j], y_train_pred[:, j])))

# MSE for the model (test data)
print("Test MSE for label {} = {}".format(j, mean_squared_error(y_test[:, j], y_test_pred[:, j])))
print("Time taken for the {}th column = {} minutes".format(j, (end-start)/60))
print()

Training R2 score for label 8 = 0.9145535247382381
Test R2 score for label 8 = 0.09866515039463752
Training MSE for label 8 = 0.0021918367122195546
Test MSE for label 8 = 0.023304269140093384
Time taken for the 8th column = 30.631388596693675 minutes



In [ ]:
j = 8
start = time()
nn_reg = MLPRegressor(hidden_layer_sizes=(2500, 500))
nn_reg.fit(X_train, y_train[:, j])

end = time()

In [ ]:
# Predictions
y_train_pred[:, j] = nn_reg.predict(X_train)
y_test_pred[:, j] = nn_reg.predict(X_test)

# R2 score for the model (training data)
print("Training R2 score for label {} = {}".format(j, r2_score(y_train[:, j], y_train_pred[:, j])))

# R2 score for the model (test data)
print("Test R2 score for label {} = {}".format(j, r2_score(y_test[:, j], y_test_pred[:, j])))

# MSE for the model (training data)
print("Training MSE for label {} = {}".format(j, mean_squared_error(y_train[:, j], y_train_pred[:, j])))

# MSE for the model (test data)
print("Test MSE for label {} = {}".format(j, mean_squared_error(y_test[:, j], y_test_pred[:, j])))
print("Time taken for the {}th column = {} minutes".format(j, (end-start)/60))
print()

Training R2 score for label 8 = 0.9433501322613781
Test R2 score for label 8 = 0.07198786572583715
Training MSE for label 8 = 0.001453158359914931
Test MSE for label 8 = 0.023994017930036225
Time taken for the 8th column = 17.28566523392995 minutes

